17102022

Бруцкий-Стемпковский

v. 01


Подготовить данные для этапа моделирования (обучения)
    
1. Анализ данных, необходимые корректировки. 
2. Обработать пропуски. 
3. Оценить выбросы. 
4. Корреляция. 
5. Тест на нормальность распределения.
6. Масштабировать и стандартизировать данные.
7. Разделите данные на тренрровачную и тестовую выборку.
8. При обработке пропусков/выбросов/скалировании тестовую и тренировачную выборку обрабатывать отдельно.

Необходимые импорты библиотек.

In [72]:
import numpy as np
import pandas as pd
import seaborn as sns

Прочитаем фрейм. Переименуем колонки. Удалим столбец Id

In [85]:
columns = [ 
            'id', 'gender', 'age', 
            'marital_status', 'job', 
            'sum_of_credit', 'credit_term',
            'tarrif_id', 'score_shk', 'edu', 
            'region', 'income', 'credit_count', 
            'overdue_credit_count', 'flag'
            ]
data = pd.read_csv('credit_train.csv', names=columns, skiprows=1, encoding='cp1251', sep=';')
data.drop(['id'], axis = 1, inplace=True)
data.head()

,gender,age,marital_status,job,sum_of_credit,credit_term,tarrif_id,score_shk,edu,region,income,credit_count,overdue_credit_count,flag
0,M,NaN,NaN,UMN,"59998,00",10,1.6,NaN,GRD,КРАСНОДАРСКИЙ КРАЙ,30000.0,1.0,1.0,0
1,F,NaN,MAR,UMN,"10889,00",6,1.1,NaN,NaN,МОСКВА,NaN,2.0,0.0,0
2,M,32.0,MAR,SPC,"10728,00",12,1.1,NaN,NaN,ОБЛ САРАТОВСКАЯ,NaN,5.0,0.0,0
3,F,27.0,NaN,SPC,"12009,09",12,1.1,NaN,NaN,ОБЛ ВОЛГОГРАДСКАЯ,NaN,2.0,0.0,0
4,M,45.0,NaN,SPC,NaN,10,1.1,"0,421385",SCH,ЧЕЛЯБИНСКАЯ ОБЛАСТЬ,NaN,1.0,0.0,0


Проверим информацию о строках.

In [86]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 170746 entries, 0 to 170745
Data columns (total 14 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   gender                170746 non-null  object 
 1   age                   170743 non-null  float64
 2   marital_status        170743 non-null  object 
 3   job                   170746 non-null  object 
 4   sum_of_credit         170744 non-null  object 
 5   credit_term           170746 non-null  int64  
 6   tarrif_id             170746 non-null  float64
 7   score_shk             170739 non-null  object 
 8   edu                   170741 non-null  object 
 9   region                170554 non-null  object 
 10  income                170741 non-null  float64
 11  credit_count          161516 non-null  float64
 12  overdue_credit_count  161516 non-null  float64
 13  flag                  170746 non-null  int64  
dtypes: float64(5), int64(2), object(7)
memory usage: 18.

Фрейм содержит 14 признаков. Как числовые, так и категориальные признаки. Целевой показатель (функция от всех остальных аргументов) зашит в последней колонке 'flag'.

Большое количество выбросов сдержится в категориях о количестве имеющихся непогашенных и просроченных кредитов. От выбросов в остальных колонка просто избавимся.

In [87]:
drop_columns = [ 
            'gender', 'age', 'marital_status', 'job', 'sum_of_credit', 'credit_term',
            'tarrif_id', 'score_shk', 'edu', 'region', 'income', 'flag'
            ]
data.dropna(subset=drop_columns, inplace=True)
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 170543 entries, 7 to 170745
Data columns (total 14 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   gender                170543 non-null  object 
 1   age                   170543 non-null  float64
 2   marital_status        170543 non-null  object 
 3   job                   170543 non-null  object 
 4   sum_of_credit         170543 non-null  object 
 5   credit_term           170543 non-null  int64  
 6   tarrif_id             170543 non-null  float64
 7   score_shk             170543 non-null  object 
 8   edu                   170543 non-null  object 
 9   region                170543 non-null  object 
 10  income                170543 non-null  float64
 11  credit_count          161331 non-null  float64
 12  overdue_credit_count  161331 non-null  float64
 13  flag                  170543 non-null  int64  
dtypes: float64(5), int64(2), object(7)
memory usage: 19.

In [81]:
data.head()

,gender,age,marital_status,job,sum_of_credit,credit_term,tarrif_id,score_shk,edu,region,income,credit_count,overdue_credit_count,flag
7,F,26.0,UNM,SPC,"47878,00",10,1.1,"0,512525",GRD,МОСКОВСКАЯ ОБЛ,60000.0,3.0,0.0,0
9,F,32.0,UNM,SPC,"26268,00",10,1.1,"0,465026",GRD,КРАЙ СТАВРОПОЛЬСКИЙ,39500.0,7.0,0.0,0
10,M,26.0,MAR,SPC,"16793,00",14,1.0,"0,445430",SCH,САНКТ-ПЕТЕРБУРГ,36000.0,2.0,0.0,0
12,M,37.0,MAR,SPC,"42289,00",10,1.6,"0,691609",SCH,ОБЛ АРХАНГЕЛЬСКАЯ,70000.0,1.0,0.0,0
13,M,43.0,MAR,INP,"57567,00",10,1.1,"0,341164",GRD,ХАНТЫ-МАНСИЙСКИЙ АО,60000.0,7.0,0.0,0


Так как с большинством столбцов мы разобрались, приведем в порядок их типы.

In [91]:
dtypes = {  "gender": "str", 
            "age": "int64",
            "marital_status": "str",
            "job": "str",
            "sum_of_credit": "float64",
            "tarrif_id": "float64",
            "score_shk": "float64",
            "edu": "str",
            "region": "object",
            "income": "float64",
            "credit_count": "int64",
            "overdue_credit_count": "int64",
            "flag": "int64"
            }

#data.astype().dtypes

In [92]:
data["region"].describe()

count             170543
unique               301
top       ОБЛ МОСКОВСКАЯ
freq               12228
Name: region, dtype: object